In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import chess, chess.pgn

In [ ]:
pgn_fn = "lichess_2025-03-07.pgn"
games = []
with open(pgn_fn) as f:
    while True:
        game = chess.pgn.read_game(f)
        if not game:
            break
        else:
            games.append(game)
len(games)